In [1]:
# Import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [2]:
# Import dataset

df = pd.read_csv('~/Marek/2_PhD/ISI/Work/data/new_isi_data.csv')
# Remove unnamed column: ,index_col=0
df.head()

,Unnamed: 0,patient_id,channel_name,resected,onset_channel,outcome,HFO_80.0-250.0,HFO_250.0-600.0,spike_rate,power_1.0-4.0,...,lin_corr_median_80.0-250.0,ren_80.0-250.0,phase_sync_80.0-250.0,coherence_80.0-250.0,lin_corr_median_250.0-600.0,ren_250.0-600.0,phase_sync_250.0-600.0,coherence_250.0-600.0,segm_type,segm_number
0,0,583,ra1,0,NON_SOZ,11.0,11,34,22,0.533375,...,0.995615,0.003025,0.985229,0.997413,0.990955,0.004121,0.970733,0.994151,0,0
1,1,583,ra2,0,NON_SOZ,11.0,16,40,49,0.518898,...,0.999169,0.000649,0.996365,0.999503,0.997006,0.001574,0.987965,0.997992,0,0
2,2,583,ra3,0,NON_SOZ,11.0,20,35,61,0.466505,...,0.999172,0.000648,0.996397,0.999509,0.996981,0.001719,0.987949,0.997969,0,0
3,3,583,ra4,0,NON_SOZ,11.0,27,37,66,0.444008,...,0.997592,0.001411,0.991613,0.998744,0.994820,0.002903,0.984822,0.997324,0,0
4,4,583,ra5,0,NON_SOZ,11.0,11,58,63,0.588096,...,0.994437,0.003369,0.981503,0.996723,0.996824,0.001831,0.987556,0.997855,0,0


In [3]:
# Drop index column

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()

,patient_id,channel_name,resected,onset_channel,outcome,HFO_80.0-250.0,HFO_250.0-600.0,spike_rate,power_1.0-4.0,power_4.0-8.0,...,lin_corr_median_80.0-250.0,ren_80.0-250.0,phase_sync_80.0-250.0,coherence_80.0-250.0,lin_corr_median_250.0-600.0,ren_250.0-600.0,phase_sync_250.0-600.0,coherence_250.0-600.0,segm_type,segm_number
0,583,ra1,0,NON_SOZ,11.0,11,34,22,0.533375,0.781969,...,0.995615,0.003025,0.985229,0.997413,0.990955,0.004121,0.970733,0.994151,0,0
1,583,ra2,0,NON_SOZ,11.0,16,40,49,0.518898,0.780688,...,0.999169,0.000649,0.996365,0.999503,0.997006,0.001574,0.987965,0.997992,0,0
2,583,ra3,0,NON_SOZ,11.0,20,35,61,0.466505,0.740345,...,0.999172,0.000648,0.996397,0.999509,0.996981,0.001719,0.987949,0.997969,0,0
3,583,ra4,0,NON_SOZ,11.0,27,37,66,0.444008,0.721173,...,0.997592,0.001411,0.991613,0.998744,0.994820,0.002903,0.984822,0.997324,0,0
4,583,ra5,0,NON_SOZ,11.0,11,58,63,0.588096,0.937518,...,0.994437,0.003369,0.981503,0.996723,0.996824,0.001831,0.987556,0.997855,0,0


In [4]:
# Drop "nan" channels across all segments

df_nans = df[df.isnull().any(axis=1)]
for pts in df_nans.patient_id.unique():
    for chns in list(df_nans.loc[df_nans.patient_id == pts,'channel_name'].unique()):
        df = df.drop(df[(df.patient_id == pts) & (df.channel_name == chns)].index)

In [5]:
# Dataframe preparation, target column 

outcome_11 = 11

df = df[df.outcome <= outcome_11]
mask = df.onset_channel == 'NON_SOZ'
df.loc[mask, 'onset_channel'] = 0
mask = df.onset_channel == 'SOZ'
df.loc[mask, 'onset_channel'] = 1
tmp = df['resected'].values * df['onset_channel'].values
df['target'] = tmp.astype(int)
df = df.rename(columns={'spike_rate':'spike_rate_0.0-0.0',
                        'pse':'pse_0.0-0.0',
                        'pac':'pac_0.0-0.0',
                        'fac':'fac_0.0-0.0',
                        'lfr':'lfr_0.0-0.0'})

features = [x for x in df if any(c.isdigit() for c in x)]
features.sort()

In [6]:
# Drop no target pts

target = 'target'
for pts in df.patient_id.unique():
    if sum(df.loc[df.patient_id == pts,target]) == 0:
        #print('patient',str(pts),'does not have target - droping this patient')
        df = df[df.patient_id != pts]

In [7]:
# Keep only segm_type = 4 

df = df.loc[df['segm_type'] == 4]
print(df.shape)
df.head()

(1959, 68)


,patient_id,channel_name,resected,onset_channel,outcome,HFO_80.0-250.0,HFO_250.0-600.0,spike_rate_0.0-0.0,power_1.0-4.0,power_4.0-8.0,...,ren_80.0-250.0,phase_sync_80.0-250.0,coherence_80.0-250.0,lin_corr_median_250.0-600.0,ren_250.0-600.0,phase_sync_250.0-600.0,coherence_250.0-600.0,segm_type,segm_number,target
34674,583,ra1,0,0,11.0,19,56,50,0.526201,0.474988,...,0.008171,0.962278,0.992315,0.951701,0.015386,0.894617,0.968141,4,0,0
34675,583,ra2,0,0,11.0,35,62,86,0.560877,0.560381,...,0.001929,0.989986,0.998410,0.982513,0.007849,0.952962,0.988048,4,0,0
34676,583,ra3,0,0,11.0,36,66,101,0.558763,0.576171,...,0.002038,0.989737,0.998386,0.982065,0.007308,0.951814,0.987657,4,0,0
34677,583,ra4,0,0,11.0,47,66,102,0.554024,0.565439,...,0.005044,0.975965,0.995656,0.975849,0.009962,0.941062,0.984107,4,0,0
34678,583,ra5,0,0,11.0,32,75,96,0.649021,0.658895,...,0.009224,0.944790,0.987954,0.980963,0.007094,0.949647,0.986945,4,0,0


In [8]:
# Number of unique patients

print(len(df.patient_id.unique()))
patients = df.patient_id.unique()
patients = np.sort(patients)
patients

18


array([  61,   63,   71,   77,   82,   89,   93,  583,  657,  717,  723,
        953,  965, 1002, 1021, 1041, 1043, 1630])

In [9]:
# Scale selected features with standard scaler (z-score)

from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder        
scaler = StandardScaler()
for pts in df.patient_id.unique(): 
    df.loc[df.patient_id == pts,features] = scaler.fit_transform(df.loc[df.patient_id == pts,features])

In [10]:
# Create a list of the columns
columns_to_keep = ['patient_id'] + [col for col in df.columns[5:65]] + ['target']

# A new DataFrame with only those columns
lazy_df = df[columns_to_keep]

In [11]:
# View lazy_df

lazy_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1959 entries, 34674 to 40452
Data columns (total 62 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   patient_id                   1959 non-null   int64  
 1   HFO_80.0-250.0               1959 non-null   float64
 2   HFO_250.0-600.0              1959 non-null   float64
 3   spike_rate_0.0-0.0           1959 non-null   float64
 4   power_1.0-4.0                1959 non-null   float64
 5   power_4.0-8.0                1959 non-null   float64
 6   power_8.0-12.0               1959 non-null   float64
 7   power_12.0-20.0              1959 non-null   float64
 8   power_20.0-45.0              1959 non-null   float64
 9   power_65.0-80.0              1959 non-null   float64
 10  power_80.0-250.0             1959 non-null   float64
 11  power_250.0-600.0            1959 non-null   float64
 12  hlx_0.0-0.0                  1959 non-null   float64
 13  hlx_1.0-4.0  

In [ ]:
# Archive legacy Lazy model 

'''
import pandas as pd
pd.set_option('display.max_colwidth', -1)
from IPython.display import display
from sklearn.model_selection import LeaveOneGroupOut
from lazypredict.Supervised import LazyClassifier
from sklearn.metrics import accuracy_score

# Define the group column
group = 'patient_id'

# Split the data into X and y
X = lazy_df.drop(columns=target)
y = lazy_df[target]

logo = LeaveOneGroupOut()
for train_index, test_index in logo.split(X, y, groups=lazy_df[group]):
    X_train, X_test = X.iloc[train_index].drop(columns=[group]), X.iloc[test_index].drop(columns=[group])
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    selected_patient_id = lazy_df.iloc[test_index][group].unique()
    print("Selected patient ID: ", selected_patient_id)
    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
    models,predictions = clf.fit(X_train, X_test, y_train, y_test)
'''

In [12]:
# Revised Lazy model, including display object, a library for handling imbalanced datasets in machine learning

import pandas as pd
pd.set_option('display.max_colwidth', -1)
from IPython.display import display
from sklearn.model_selection import LeaveOneGroupOut
from lazypredict.Supervised import LazyClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

# Define the group column
group = 'patient_id'

# Split the data into X and y
X = lazy_df.drop(columns=target)
y = lazy_df[target]

logo = LeaveOneGroupOut()

for train_index, test_index in logo.split(X, y, groups=lazy_df[group]):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    selected_patient_id = lazy_df.iloc[test_index][group].unique()
    print("Selected patient ID: ", selected_patient_id)
    print("X_train shape: ", X_train.shape)
    print("X_test shape: ", X_test.shape)
    print("y_train shape: ", y_train.shape)
    print("y_test shape: ", y_test.shape)
    # oversampling the minority class using SMOTE
    sm = SMOTE(sampling_strategy='auto')
    X_train, y_train = sm.fit_resample(X_train, y_train)
    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
    models,predictions = clf.fit(X_train.drop(columns=[group]), X_test.drop(columns=[group]), y_train, y_test)
    display(models.style.set_properties(**{'font-size': '9pt'}))


Selected patient ID:  [61]
X_train shape:  (1809, 61)
X_test shape:  (150, 61)
y_train shape:  (1809,)
y_test shape:  (150,)


100%|███████████████████████████████████████████| 29/29 [00:08<00:00,  3.44it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.813333,0.848700,0.848700,0.859006,0.060872
RidgeClassifier,0.813333,0.848700,0.848700,0.859006,0.028867
CalibratedClassifierCV,0.833333,0.807329,0.807329,0.871500,1.357625
LinearSVC,0.820000,0.800236,0.800236,0.862496,0.386101
BaggingClassifier,0.860000,0.717494,0.717494,0.885972,1.206248
NuSVC,0.853333,0.713948,0.713948,0.881586,0.400160
NearestCentroid,0.733333,0.702128,0.702128,0.802233,0.013656
RandomForestClassifier,0.913333,0.693853,0.693853,0.919058,1.349251
LogisticRegression,0.806667,0.689125,0.689125,0.850940,0.055938


Selected patient ID:  [63]
X_train shape:  (1839, 61)
X_test shape:  (120, 61)
y_train shape:  (1839,)
y_test shape:  (120,)


100%|███████████████████████████████████████████| 29/29 [00:08<00:00,  3.35it/s]


...

100%|███████████████████████████████████████████| 29/29 [00:09<00:00,  3.13it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
PassiveAggressiveClassifier,0.549296,0.694444,0.694444,0.618220,0.020264
AdaBoostClassifier,0.732394,0.656810,0.656810,0.770122,1.062137
NuSVC,0.788732,0.641577,0.641577,0.807258,0.420869
SGDClassifier,0.676056,0.624552,0.624552,0.727390,0.052601
LogisticRegression,0.718310,0.601254,0.601254,0.756257,0.042193
SVC,0.774648,0.586022,0.586022,0.791379,0.238345
DecisionTreeClassifier,0.845070,0.578853,0.578853,0.831407,0.268785
NearestCentroid,0.676056,0.577061,0.577061,0.725010,0.014082
ExtraTreeClassifier,0.718310,0.553763,0.553763,0.751933,0.013292


Selected patient ID:  [1630]
X_train shape:  (1788, 61)
X_test shape:  (171, 61)
y_train shape:  (1788,)
y_test shape:  (171,)


100%|███████████████████████████████████████████| 29/29 [00:08<00:00,  3.48it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
PassiveAggressiveClassifier,0.742690,0.863354,0.863354,0.810778,0.021298
LinearDiscriminantAnalysis,0.883041,0.844099,0.844099,0.905974,0.052627
RidgeClassifierCV,0.877193,0.840994,0.840994,0.901983,0.029590
RidgeClassifier,0.877193,0.840994,0.840994,0.901983,0.015493
SGDClassifier,0.789474,0.794410,0.794410,0.842636,0.044540
LGBMClassifier,0.877193,0.794099,0.794099,0.900509,0.151831
XGBClassifier,0.877193,0.794099,0.794099,0.900509,0.332430
LogisticRegression,0.812865,0.759938,0.759938,0.857533,0.042060
KNeighborsClassifier,0.707602,0.750932,0.750932,0.785132,0.016056


Explanation:

1) This code uses the following libraries:

pandas: a library for data manipulation and analysis
sklearn (Scikit-learn): a library for machine learning in Python
IPython.display: a library for creating display objects for the Jupyter notebook
lazypredict: a library for making machine learning easier and faster
imblearn: a library for handling imbalanced datasets in machine learning.


2) The "group" variable:

In this code represents the "patient_id" column in the dataset and is used as a grouping variable in the LeaveOneGroupOut cross-validation method. 


3) LeaveOneGroupOut (LOGO)

Is a cross-validation method used in machine learning to evaluate the performance of a model. The idea behind LOGO is to perform leave-one-out cross-validation (LOOCV) on group labels, instead of data samples. In this method, the data is divided into groups, and in each iteration, one entire group is left out for testing, and the rest of the groups are used for training. This way, the model is trained on data from all but one group, and then evaluated on the left-out group.The method "logo.split" is used to perform the cross-validation by splitting the data into training and test sets.

More practical:

The code is using the Leave-One-Group-Out cross-validation strategy, where each patient is considered a group. For each iteration of the loop, one patient is selected for testing and the remaining patients are used for training the model. In your example, if patient 61 is selected for testing, the model is trained on the data from patients 63, 71, 77, 82, 89, 93, 583, 657, 717, 723, 953, 965, 1002, 1021, 1041, 1043, and 1630. This is repeated for all patients, so each patient is used once for testing and all other patients are used for training.


4) SMOTE (Synthetic Minority Over-sampling Technique)

Is a popular oversampling method used to handle imbalanced datasets in machine learning. Imbalanced datasets occur when the number of samples belonging to one class is much higher than the number of samples belonging to the other class. This can result in a biased model that performs poorly on the minority class. SMOTE works by creating synthetic samples of the minority class instead of simply duplicating the existing samples. The algorithm does this by selecting a sample from the minority class and finding its k nearest neighbors. Then, it generates synthetic samples by interpolating between the selected sample and its neighbors. This process helps to create more diverse and representative samples for the minority class and thus helps to balance the class distribution.
In this code, the SMOTE method is used to oversample the minority class in the training data. The SMOTE object "sm" is created using the "SMOTE" class from the imblearn library, with the sampling_strategy set to 'auto'. The "fit_resample" method is then used to apply the SMOTE method to the training data. The result is a balanced training set with an equal number of samples from each class.


5) An explanation of each evaluation metric:

Accuracy: The proportion of correctly classified samples out of the total number of samples. It is a simple and commonly used metric to evaluate the performance of a classifier. However, it may not be suitable when the classes are imbalanced.

Balanced Accuracy: Similar to accuracy, but takes into account the imbalance of the classes. It is the average of sensitivity (true positive rate) and specificity (true negative rate).

ROC AUC: Receiver Operating Characteristic (ROC) curve is a plot of the true positive rate (sensitivity) versus the false positive rate (1 - specificity) for different classification thresholds. Area Under the Curve (AUC) of the ROC curve is a metric that measures the performance of the classifier across all possible thresholds. It is a useful metric when the classes are imbalanced, and it gives an idea of how well the classifier can discriminate between positive and negative samples.

F1 Score: The harmonic mean of precision and recall. It is a metric that balances the importance of precision (the proportion of true positives out of all predicted positives) and recall (the proportion of true positives out of all actual positives). It is a good metric when the classes are imbalanced, and it gives a balanced view of the classifier's performance.
